In [4]:
import torch
import pyaudio
import wave
import os
import time
from AudioUtil import AudioUtil
from AudioClassifier import AudioClassifier

print(torch.__version__)
model = AudioClassifier()
state_dict = torch.load("data/models/V8_model_state_dict.pth")
model.load_state_dict(state_dict)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model = model.to(device)
model.eval()

CHUNK = 1050
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100

2.3.0+cu118
Using device: cuda:0


In [9]:
def record_audio(duration=2):
    frames = []
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)
    try:
        # Calculate the correct number of iterations to cover the duration
        num_frames = int((RATE / CHUNK) * duration)
        for _ in range(num_frames):
            data = stream.read(CHUNK)
            frames.append(data)
    finally:
        stream.stop_stream()
        stream.close()
        p.terminate()
    return frames

def save_temp_audio(frames, filename="temp_audio.wav"):
    with wave.open(filename, 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(pyaudio.PyAudio().get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))

def process_audio_file(filename):
    class_names = {0: 'Noise', 1: 'Knock'} 
    aud = AudioUtil.open(filename)
    # os.remove(filename)
    # reaud = AudioUtil.resample(aud, RATE)
    # rechan = AudioUtil.rechannel(reaud, CHANNELS)
    # dur_aud = AudioUtil.pad_trunc(rechan, 2000)
    rechan = AudioUtil.rechannel(aud, 2)
    sgram = AudioUtil.spectro_gram(rechan, n_mels=64, n_fft=1024, hop_len=None)
    # sgram = AudioUtil.spectro_gram(aud, n_mels=64, n_fft=1024, hop_len=None)
    sgram = sgram.to(device)

    outputs = model(sgram.unsqueeze(0))
    print(outputs)

    # Get the predicted class with the highest score
    _, prediction = torch.max(outputs, 1)
    predicted_classes = [class_names.get(p.item(), p.item()) for p in prediction][0]
    # print(prediction.item())
    print(predicted_classes)
    # if prediction > 0.5:
    #     print("Klopfen erkannt!")

In [11]:
# process_audio_file for every file in train
for file in os.listdir("data/validation"):
    print(f"Processing {file}")
    process_audio_file(f"data/validation/{file}")
# process_audio_file("data/knocks/knock_10.wav")
# process_audio_file("data/noises/noise_72.wav")

Processing knock_10.wav
tensor([[1.8362e-36, 1.0000e+00]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
Knock
Processing knock_10_var2.wav
tensor([[0., 1.]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
Knock
Processing knock_10_var4.wav
tensor([[2.8026e-45, 1.0000e+00]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
Knock
Processing knock_11_var0.wav
tensor([[6.6982e-43, 1.0000e+00]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
Knock
Processing knock_11_var1.wav
tensor([[3.3533e-42, 1.0000e+00]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
Knock
Processing knock_1_var0.wav
tensor([[1.5871e-36, 1.0000e+00]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
Knock
Processing knock_1_var7.wav
tensor([[2.2346e-36, 1.0000e+00]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
Knock
Processing knock_2_var5.wav
tensor([[8.1184e-40, 1.0000e+00]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
Knock
Processing knock_2_var9.wav
tensor([[4.0231e-42, 1.0000e+00]], device='cuda:0', grad_fn=<SoftmaxBackward

In [10]:
print("Starte Klopfen-Erkennung...")
while True:
    frames = record_audio()
    save_temp_audio(frames)
    process_audio_file("temp_audio.wav")

Starte Klopfen-Erkennung...
tensor([[3.2408e-26, 1.0000e+00]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
Knock
tensor([[1.1703e-26, 1.0000e+00]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
Knock
tensor([[3.0072e-19, 1.0000e+00]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
Knock
tensor([[4.2827e-15, 1.0000e+00]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
Knock


KeyboardInterrupt: 